# Data Analysis
The data analysis is crucial for the understanding of the raw counter/KPI dataset. It includes some data pre-processing (removing invalid entries, missing data handling, etc.) as well as analysis on the counter/KPIs statistics.

In [32]:
import numpy as np
import pandas as pd
import scipy
from pandas import DataFrame
from pandas import read_csv
import matplotlib.pyplot as plt

In [34]:
a = pd.read_csv("data/data_wide_min_max.csv", low_memory=False)

In [41]:
a['User_THP_DL_kbps']

0                  12/18/2018
1                           0
2                         NaN
3         0.16597813033539074
4         0.18917366324749751
5          0.2956798128111791
6           0.649423951404181
7         0.12399756110890493
8          0.2694785531892605
9         0.42967878597881326
10         0.4953081238983578
11         0.3119579702849906
12          0.270747998056189
13        0.28248473025714477
14         0.3041228576485263
15         0.2396150000805766
16         0.4526777197995158
17         0.5899874161709641
18         0.4488420454229167
19         0.7380083970716415
20          0.355469979114469
21         0.4979246366137965
22         0.5697995333699827
23        0.28252649554280995
24        0.32428408760273464
25        0.17405559153735956
26         0.4591175421688237
27         0.3544068877313205
28        0.33242254810350386
29         0.2607678807977167
                 ...         
18408       0.252907077015136
18409     0.28275153349649274
18410     

In [2]:
df_raw = pd.read_csv("data/common_files_HighLoadAnalysis201812200038255c1a038136d32.csv", encoding = 'gbk')

df_clean =  df_raw[['day', 'hour', 'location','Users_RRC_Avg', 'DL_PDSCH_Usage', 'PDCCH_CCE_利用率超过70%的比例',
       'DL_Volume_GB', 'UL_PUSCH_Usage', 'UL_Volume_GB', 'RLC时延',
       'LTE_User_THP_DL_kpbs', 'LTE_User_THP_UL_kpbs', 'CellTputDl_kbps',
       'Uplink Cell Throughput(kbps)', 'MAC层下行BLER', 'MAC层上行BLER',
       'RLC层下行重传率(%)', 'RLC层上行重传率(%)', 'SRfail占SR的比例']].copy()

df_clean.columns = ['day','hour','cell', 'Users_RRC_Avg', 'PDSCH_Usage_DL',
       'PDCCH_CCE_Above_70', 'DL_Volume_GB', 'PUSCH_Usage_UL', 'UL_Volume_GB',
       'RLC_Time_Delay', 'User_THP_DL_kbps', 'User_THP_UL_kbps',
       'Cell_THP_Dl_kbps', 'Cell_THP_UL_kbps',
       'MAC_Retransmission_DL', 'MAC_Retransmission_UL', 'RLC_Retransmission_DL', 'RLC_Retransmission_UL','SRfail_RAatt']

# 删除RLC重传率相关的两个指标
df_clean.drop(['RLC_Retransmission_DL', 'RLC_Retransmission_UL'], axis=1, inplace=True)


## 缺失值处理

In [6]:
print("# of df_clean before dropna:", len(df_clean))
df_clean.dropna(inplace = True)
print("# of df_clean after dropna:", len(df_clean))

# of df_clean before dropna: 585214
# of df_clean after dropna: 585214


## 数据归一化

In [4]:
df_normal = df_clean.copy()

# modify the num of days according to days of data
day = 1
df_normal = df_normal.groupby(['cell']).filter(lambda group: len(group) == 24 * day)
df_normal = df_normal.sort_values(by=['cell','day','hour'])

kpi_list = ['Users_RRC_Avg', 'PDSCH_Usage_DL',
       'PDCCH_CCE_Above_70', 'DL_Volume_GB', 'PUSCH_Usage_UL', 'UL_Volume_GB',
       'RLC_Time_Delay', 'User_THP_DL_kbps', 'User_THP_UL_kbps',
       'Cell_THP_Dl_kbps', 'Cell_THP_UL_kbps',
       'MAC_Retransmission_DL', 'MAC_Retransmission_UL',
       'SRfail_RAatt']
# 'RLC_Retransmission_DL', 'RLC_Retransmission_UL'

# pdf_90归一化
# kpi_list_90pdf = ['Users_RRC_Avg', 'DL_Volume_GB', 'UL_Volume_GB']
df_normal_use_pdf90 = df_normal.copy()
def getmax(x,y):
    if x<y:
        return 0
    else:
        return (x-y)/x
for kpi in kpi_list:
    if kpi not in ['RLC_Retransmission_DL', 'RLC_Retransmission_UL', 'SRfail_RAatt', 'PDCCH_CCE_Above_70']:
        base = np.percentile(df_normal_use_pdf90[kpi],90)
        df_normal_use_pdf90[kpi] = df_normal_use_pdf90[kpi].apply(lambda x:getmax(x,base))
#     df_normal_use_pdf90[kpi] = (df_normal_use_pdf90[kpi] - base) / df_normal_use_pdf90[kpi]

# min-max 归一化
# max设为大于均值的三倍标准差
df_normal_use_minmax = df_normal.copy()
max_use = df_normal[kpi_list].mean() + 3 * df_normal[kpi_list].std()
min_use = df_normal[kpi_list].min()
df_normal_use_minmax[kpi_list] = (df_normal_use_minmax[kpi_list] - min_use) / (max_use - min_use)

# 保存归一化时使用的min max值
# df_max = max_use.to_frame(name='MAX')
# df_min = max_use.to_frame(name='MIN')
# df_min.merge(df_max, left_index=True, right_index=True).to_csv("data/min_max_used.csv")

In [31]:
# 保存归一化时使用的min max值
df_max_actual = df_normal[kpi_list].max().to_frame(name="MAX_ACTUAL")
df_min_actual = df_normal[kpi_list].min().to_frame(name="MIN_ACTUAL")
df_mean = df_normal[kpi_list].mean().to_frame(name="MEAN")
df_max_used = max_use.to_frame(name='MAX_USED')
df_min_used = min_use.to_frame(name='MIN_USED')
df_mean.join(df_min_actual).join(df_max_actual).join(df_min_used).join(df_max_used).to_csv("data/min_max_used.csv")
# df_min_actual.merge(df_max_actual, left_index=True, right_index=True)

# df_min_used.merge(df_max_used, left_index=True, right_index=True).to_csv("data/min_max_used.csv")

## 生成wide table并储存

In [5]:
nb_kpis = len(kpi_list)
nb_hours = 24
nb_features = nb_kpis * nb_hours

def gen_save_wide_table(df_normal, name):
    """
    generate wide table per given datasets after normalization
    """
    df_normal_wide = pd.DataFrame(df_normal.groupby(['cell', 'day', 'hour']).sum().unstack(['day','hour']))
    df_normal_wide.to_csv("data/data_wide_{}.csv".format(name))

gen_save_wide_table(df_normal_use_minmax, name="min_max")
gen_save_wide_table(df_normal_use_pdf90, name="pdf90")